# What is LangChain?

<img align="left" width="200" alt="RagChart" src="https://media.licdn.com/dms/image/D4E12AQHQP9J275Q_uA/article-cover_image-shrink_600_2000/0/1700940849777?e=2147483647&v=beta&t=m0HEQrukIOqU4fe1K9M19PaHq3UbvEubLzeIH1shcSc" style="margin: 0px 15px 0px 0px;"/>

LangChain is an open-source Python library that enables anyone who can write code to build LLM-powered applications such as Chatbots, Presonal Asistants, Summarization, Analysis, etc.

### Using HuggingFace and OpenAI LLM Models



In [25]:
# OpenAI -> Needs payment
from langchain_openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo-instruct")

'''
# HuggingFace -> Free to use
from langchain import HuggingFaceHub
llm = HuggingFaceHub(repo_id = "google/flan-t5-large")
'''

prompt = "Alice has a parrot. What animal is Alice's pet?"
completion = llm(prompt)

print(completion)



A parrot.


### Text Emfedding with OpenAI and HuggingFace

The embeddings model takes a text as an input and outputs a list of floats (embeddings).

In [26]:

# Open AI -> Needs payment 
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

'''
# HugingFace -> Free to use
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
'''

text = "Alice has a parrot. What animal is Alice's pet?"
text_embedding = embeddings.embed_query(text)

print(text_embedding)

[-0.03506468305293723, 0.029810185198174816, -0.004789528996659525, -0.0018111117306932115, -0.019938487951036876, 0.00023126628820376068, -0.008720649577538472, -0.05545839031151321, -0.02874367740562338, 0.03319179331861247, 0.005420329155654441, 0.01800056582269499, -0.012160786666327014, 0.0008149190267364806, -0.012453425920145906, -0.01666092719067817, -0.014644968610954483, 0.03527278522367587, 0.027547106687682888, 0.015763501014867985, -0.046354056314796395, 0.02014658639648514, 0.027963305441224606, -0.0241394867272255, 0.023996418813149168, 0.02177235899400944, 0.02099198889225585, 0.014749017833678615, -0.01997750664238907, -0.002880870886831791, -0.01770142314320987, 0.010931701614023149, 0.04580779463586562, 0.016699945882030366, -0.00684775844622982, 0.005192720898868781, -0.01930118390071443, -0.0797539518041475, 0.012616003179898336, 0.013604473589745376, -0.021616286091245832, 0.013500423435698652, -0.05368954793726739, -0.01914510913530564, -0.013253306298898187, -0.

### Creating Prompt Templates

In [27]:
from langchain import PromptTemplate

# Zero-Shot Prompting
template = "What is a good name for a company that makes {product}?"

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [28]:
from langchain import PromptTemplate, FewShotPromptTemplate

# Few-Shot Prompting
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

example_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n",
)

few_shot_prompt.format(input="big")

'Give the antonym of every input\n\nWord: happy\nAntonym: sad\n\n\n\nWord: tall\nAntonym: short\n\n\nWord: big\nAntonym:'

### Chaining: Combines LLMs with other compenents

In [29]:
# 1) Combining LLMs with prompt templates
from langchain.chains import LLMChain, SimpleSequentialChain

# First Chain
chain1 = LLMChain(llm = llm, 
                  prompt = prompt)
# use prompt template we creatd befoe with llm model

# Run the chain only specifying the input variable.
chain1.run("colorful sock")

'\n\n"Rainbow Toes Co." '

In [30]:
# 2) Combining multiple LLMs sequentially by taking 
# the first LLM's output as the input for the second LLM
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)

# Create a second chain
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Combine the first and the second chain 
overall_chain = SimpleSequentialChain(chains=[chain1, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("colorful socks")



> Entering new SimpleSequentialChain chain...


"Rainbow Threads"


"Unleash your true colors with Rainbow Threads!"

> Finished chain.


### Indexes: Accesing external data

It provides accessing to some specific documents or emails. For this purpose we need to load necessary data by emmbedding and then store in a vectore database

In [31]:
from langchain.document_loaders import YoutubeLoader
from langchain.vectorstores import FAISS

loader = YoutubeLoader.from_youtube_url("https://youtu.be/LNHBMFCzznE?si=Lw5BJb3oxtccRucY")
    
documents = loader.load()

# create the vectorestore to use as the index
db = FAISS.from_documents(documents, embeddings)

documents

[Document(page_content='Translator: Jessica Lee\nReviewer: Denise RQ So how do we learn? And why does some of us learn things\nmore easily than others? So, as I just mentioned,\nI\'m Dr. Lara Boyd. I am a brain researcher here\nat the University of British Columbia. These are the questions that fascinate me. (Cheers) (Applause) So brain research\nis one of the great frontiers in the understanding of human physiology, and also in the consideration\nof what makes us who we are. It\'s an amazing time\nto be a brain researcher, and I would argue to you that I have the most interesting job\nin the world. What we know about the brain\nis changing at a breathtaking pace. And much of what we thought we knew\nand understood about the brain turns out to be not true or incomplete. Some of these misconceptions\nare more obvious than others. For example, we used to think that after childhood the brain did not,\nreally could not change. And it turns out that nothing\ncould be farther from the truth.

We collected external datas from youtube video and store it in vectore dataset. Let's use it question-answering task with an retriever.

In [32]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True)

query = "Does our brain change?"
result = qa({"query": query})

print(result['result'])

 Yes, our brain is constantly changing through a process called neuroplasticity. This means that every time we learn something new or practice a skill, our brain physically changes and forms new connections between neurons. These changes can occur in chemical, structural, and functional ways, and they can happen at any age. Additionally, these changes can support recovery after brain damage or injury. However, the brain can also change in negative ways, such as forgetting something we once knew or developing an addiction. Overall, our brains are highly plastic and are constantly being shaped by our behaviors and experiences.


### Memory: Remembering previous conversations 

It is essential for some applications like chatbots that they can remember previous conversation. But, LLMs don't have any long-term memory unless you input the chat history.

In [33]:
from langchain import ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Alice has a parrot.")
conversation.predict(input="Bob has two cats.")
conversation.predict(input="How many pets do Alice and Bob have?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Alice has a parrot.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Alice has a parrot.
AI:  That's interesting! Parrots are very intelligent birds, known for their ability to mimic human speech and solve puzzles. They also have vibrant colors and can live for decades. Does Alice have any other pets?
Human: Bob has two cats.
AI:

> Finished chain.


> Entering n

' From the information provided, it seems that Alice has one parrot and Bob has two cats, so together they have three pets.'

### Agents: Accesing other tools

LLMs become outdated quickly, are bad at math and also lack contextual information. Because of that, they can hallucinate on tasks they can't accomplısh on their own. So we need to give them access to supplementary tools such as search engines, calculators, and lookups. Of course we need to also develop an agent that makes decision about which tools to use to acomplish a task based on the LLM's output.

In [34]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)


agent.run("When was Barack Obama born? How old was he in 2022?")



> Entering new AgentExecutor chain...
 I need to find out when Barack Obama was born and how old he would be in 2022.
Action: wikipedia
Action Input: Barack Obama
Observation: Page: Barack Obama
Summary: Barack Hussein Obama II ( , bə-RAHK hoo-SAYN oh-BAH-mə; born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American  president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008, as an Illinois state senator from 1997 to 2004, and as a community service organizer, civil rights lawyer, and university lecturer. 
Obama was born in Honolulu, Hawaii. He graduated from Columbia University in 1983 with a B.A. in political science and later worked as a community organizer in Chicago. In 1988, Obama enrolled in Harvard Law School, where he was the first black president of the Harvard Law Review. He became a civil rights

'Barack Obama was born on August 4, 1961 and he would be 61 years old in 2022.'